<a href="https://colab.research.google.com/github/Fahad-Blog/Data-Science-Portfolio/blob/main/Data_Analysis_with_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install Gemini Libraries
!pip install -U google-genai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.


In [ ]:
pip install pymongo sentence_transformers pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 15.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import os
from google import genai
from google.colab import userdata

# --- SETUP ---
api_key = userdata.get('gemini_api_key')
client = genai.Client(api_key=api_key)

# 1. Load and Optimize Data
# We use Parquet for scalability (10k+ rows)
EXCEL_FILE = "/content/drive/MyDrive/Colab Notebooks/Orders.xlsx"
PARQUET_FILE = "data.parquet"

if not os.path.exists(PARQUET_FILE):
    print("🚀 Initializing Data: Converting Excel to Parquet...")
    df_raw = pd.read_excel(EXCEL_FILE)
    df_raw.to_parquet(PARQUET_FILE)

# Load the fast parquet file into memory
df = pd.read_parquet(PARQUET_FILE)

def get_data_insight(user_query):
    # Step A: Generate the Schema Context
    schema_context = f"""
    You have a DataFrame named 'df'.
    Columns: {df.columns.tolist()}
    Data Types: {df.dtypes.to_dict()}
    Sample Data (First 2 rows):
    {df.head(2).to_string()}
    """

    # Step B: LLM writes the Python code
    code_prompt = f"""
    {schema_context}

    Task: Write ONLY a one-line Python pandas expression to answer the question.
    Question: {user_query}

    Rules:
    - No prose, no explanations.
    - Return only the code.
    - Use 'df' as the variable.
    Example: df['sales'].sum()
    """

    print("🤖 AI is writing analysis code...")
    code_response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=code_prompt
    )

    python_code = code_response.text.strip().replace('```python', '').replace('```', '')

    try:
        # Step C: EXECUTE the code safely
        # We pass 'df' and 'pd' into the eval environment
        result = eval(python_code, {"df": df, "pd": pd})

        # Step D: LLM interprets the result for the final response
        interpretation_prompt = f"""
        User Question: {user_query}
        Analysis Code Used: {python_code}
        Raw Result from Data: {result}

        Task: Provide a professional, concise, and helpful summary of this result for a business user.
        """

        print("🧠 AI is interpreting the result...")
        final_response = client.models.generate_content(
            model='gemini-2.5-flash',
            contents=interpretation_prompt
        )

        return final_response.text

    except Exception as e:
        return f"⚠️ Calculation Error: The AI tried to run `{python_code}` but failed. Error: {e}"

# --- TEST THE SYSTEM ---
print("-" * 30)
query = "Find the top 3 largest order id with maximum profit.?"
answer = get_data_insight(query)
print(f"\nFinal Response:\n{answer}")

------------------------------
🤖 AI is writing analysis code...
🧠 AI is interpreting the result...

Final Response:
The top 3 Order IDs that generated the maximum profit are:

*   **400**
*   **2721**
*   **1460**

These are your most profitable individual orders.
